In [1]:
## Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3    ## SQL Interface
import pickle     ## Used to save your data - Converts objects to byte stream and vice versa

from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model

import gensim    ## To build Word2Vec model
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

C:\Users\Pratheesh\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from sklearn.naive_bayes import MultinomialNB    #Multinomial Naive - Bayes

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


In [3]:
conn = sqlite3.connect('final.sqlite')  #Loading the sqlite file for future use
final = pd.read_sql_query("""SELECT * FROM Reviews""", conn)
conn.close()
final.drop(['index'],axis=1,inplace = True)
final.head()

,SEM 1 SGPA,SEM 1 KT,SEM 2 SGPA,SEM 2 KT,SEM 3 SGPA,SEM 3 KT,SEM 4 SGPA,SEM 4 KT,SEM 5 SGPA,SEM 5 KT,...,2_hrs_lect,Submissions,5_hrs_lect,5_hrs_pracs,Coaching_classes,Teacher's Feedback,Label,Scocial_Skills,Average pointer,Cleaned_Feedback
0,7.10,0,6.85,0,7.20,1.0,7.30,0.0,7.60,0,...,0,0,0,0,0,Disciplined and hard working,1,0,7.13125,disciplin hard work
1,7.20,0,7.10,0,6.11,0.0,6.67,0.0,7.14,0,...,1,1,0,0,1,Good leadership skills,1,1,7.05375,good leadership skill
2,3.80,4,4.20,2,4.20,1.0,3.80,2.0,4.35,0,...,0,1,0,0,0,Very talkative and Poor attendance,0,0,4.05875,talkat poor attend
3,6.91,1,6.60,2,5.82,1.0,6.10,0.0,7.71,0,...,0,1,0,1,0,Respectful to Authority and Others,1,0,6.50500,respect author other
4,3.50,1,3.40,2,4.00,2.0,4.10,2.0,3.63,0,...,0,0,0,0,0,Bunk lectuer,0,0,3.81500,bunk lectuer


In [7]:
final.shape

(262, 32)

In [4]:
final.columns

Index(['SEM 1 SGPA', 'SEM 1 KT', 'SEM 2 SGPA', 'SEM 2 KT', 'SEM 3 SGPA',
       'SEM 3 KT', 'SEM 4 SGPA', 'SEM 4 KT', 'SEM 5 SGPA', 'SEM 5 KT',
       'SEM 6 SGPA', 'SEM 6 KT', 'SEM 7 SGPA', 'SEM 7 KT', 'SEM 8 SGPA',
       'Hours_On_Assignment', 'Hours_On_Studies', 'Travel_Time', 'Attendance',
       'Internet_Availability', 'Internet_Speed', 'Mode_Of_Transportation',
       '2_hrs_lect', 'Submissions', '5_hrs_lect', '5_hrs_pracs',
       'Coaching_classes', 'Teacher's Feedback', 'Label', 'Scocial_Skills',
       'Average pointer', 'Cleaned_Feedback'],
      dtype='object')

In [5]:
final.drop(['SEM 5 SGPA','SEM 5 KT','SEM 6 SGPA', 'SEM 6 KT', 'SEM 7 SGPA', 'SEM 7 KT', 'SEM 8 SGPA',"Teacher's Feedback","Cleaned_Feedback",'Average pointer'],axis=1,inplace = True)
final.head()

,SEM 1 SGPA,SEM 1 KT,SEM 2 SGPA,SEM 2 KT,SEM 3 SGPA,SEM 3 KT,SEM 4 SGPA,SEM 4 KT,Hours_On_Assignment,Hours_On_Studies,...,Internet_Availability,Internet_Speed,Mode_Of_Transportation,2_hrs_lect,Submissions,5_hrs_lect,5_hrs_pracs,Coaching_classes,Label,Scocial_Skills
0,7.10,0,6.85,0,7.20,1.0,7.30,0.0,1,0,...,1,0,1,0,0,0,0,0,1,0
1,7.20,0,7.10,0,6.11,0.0,6.67,0.0,0,0,...,1,0,1,1,1,0,0,1,1,1
2,3.80,4,4.20,2,4.20,1.0,3.80,2.0,0,0,...,1,0,1,0,1,0,0,0,0,0
3,6.91,1,6.60,2,5.82,1.0,6.10,0.0,1,0,...,1,0,1,0,1,0,1,0,1,0
4,3.50,1,3.40,2,4.00,2.0,4.10,2.0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [6]:
final.columns

Index(['SEM 1 SGPA', 'SEM 1 KT', 'SEM 2 SGPA', 'SEM 2 KT', 'SEM 3 SGPA',
       'SEM 3 KT', 'SEM 4 SGPA', 'SEM 4 KT', 'Hours_On_Assignment',
       'Hours_On_Studies', 'Travel_Time', 'Attendance',
       'Internet_Availability', 'Internet_Speed', 'Mode_Of_Transportation',
       '2_hrs_lect', 'Submissions', '5_hrs_lect', '5_hrs_pracs',
       'Coaching_classes', 'Label', 'Scocial_Skills'],
      dtype='object')

In [7]:
final.shape

(262, 22)

# Logistic Regression

In [29]:
# Values for the hyperparameter 'C':
tuned_params = [{'C': [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]}]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)

In [30]:
# Grid Search
model = GridSearchCV(LogisticRegression(), tuned_params, scoring = 'accuracy')
model.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [31]:
print(model.best_estimator_)
print(model.score(X_test, y_test))

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
1.0


In [34]:
clf = LogisticRegression(C = 0.01)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)*float(100)
acc

100.0

In [39]:
X_test[0]

array([ 4.64,  1.  ,  4.21,  1.  ,  4.31,  1.  ,  4.19,  1.  ,  0.  ,
        0.  ,  2.  ,  5.  ,  1.  ,  0.  ,  3.  ,  0.  ,  1.  ,  0.  ,
        0.  ,  0.  ,  0.  ])

In [40]:
X

array([ 6.5 ,  0.  ,  7.22,  0.  ,  7.65,  0.  ,  8.55,  0.  ,  1.  ,
        1.  ,  0.  ,  1.  ,  1.  ,  0.  ,  1.  ,  0.  ,  1.  ,  1.  ,
        1.  ,  1.  ,  1.  ])

In [46]:
name = 'Pratheesh'
gender = 1
sem1_ptr = 4.6 
sem1_kt = 2
sem2_ptr = 5.2
sem2_kt = 1
sem3_ptr = 4.5
sem3_kt = 4
sem4_ptr = 5.5
sem4_kt = 1
assign = 0
travel = 1
studies = 0
attnd = 0
internet = 1
int_speed = 0
mode_trns = 1
lectures_2 = 0
submissions = 0
lectures_5 =  0
practicals_5 = 0
coaching_class = 1 
social_skills = 2
X = np.array([sem1_ptr,sem1_kt,sem2_ptr,sem2_kt,sem3_ptr,sem3_kt,sem4_ptr,sem4_kt,assign,travel,studies,attnd,internet,int_speed,mode_trns,lectures_2,submissions,lectures_5,practicals_5,coaching_class,social_skills])
y_pred = clf.predict([X])
print(y_pred)

[1]


In [35]:
confusion_matrix(y_test, y_pred).T

array([[25,  0],
       [ 0, 53]], dtype=int64)